In [1]:
from aiida import load_profile
aiida_profile = load_profile()
aiida_profile.name

'default'

In [2]:
## Loading the some require packages and module
from aiida_kkr.workflows import (combine_imps_wc, kkr_flex_wc,
                                 kkr_imp_wc, kkr_imp_sub_wc
                                )
from aiida_kkr.calculations import KkrCalculation, KkrimpCalculation
from aiida.orm import (Group, load_group, load_node, Dict, Code
                      )
from aiida_kkr.tools import kkrparams
from aiida.engine import submit

In [3]:
metadata_option_1 = {'max_wallclock_seconds': 36000,'resources': 
               {'tot_num_mpiprocs': 48, 'num_machines': 1},
              'custom_scheduler_commands': 
              '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
              'withmpi': True
                    }

In [4]:
# Note that here for doped 3d atoms into the Bi2Te3 host

group_74 = load_group(74)
## first node of group-64 is for kkr_imp_wc process
# is being used for the host_gf or kkr_flex files

imp_wc_with_gf = load_node(16100)
imp1_wc = group_74.nodes[1]
imp2_wc = group_74.nodes[2]

# gf_host_remote = imp_wc_with_gf.inputs.remote_data_gf
# host_gf_wc = imp_wc_with_gf.get_outgoing(node_class=kkr_flex_wc).all()[0].node


impurity1_output_node = imp1_wc.outputs.workflow_info
impurity2_output_node = imp2_wc.outputs.workflow_info
offset_imp2 = {'index':1}

In [5]:
# kkr_code_iffslurm = kkrflex@iffslurm

In [6]:
scf_node = imp1_wc.get_outgoing(node_class=kkr_imp_sub_wc).all()[0].node
## kkr_flex_wc
host_gf = imp1_wc.inputs.remote_data_gf.get_incoming(node_class=kkr_flex_wc).all()[0].node
kkr_calc = host_gf.get_outgoing(node_class=KkrCalculation).all()[0].node
kkr_flex_kkr_param = kkr_calc.inputs.parameters

In [7]:
kkr_flex_kkr_param = kkr_calc.inputs.parameters.get_dict()
kkr_flex_kkr_param['RCLUSTZ'] *= 1
kkr_flex_kkr_param['NSHELD'] = 2500

In [8]:
# kkr_flex_kkr_param

In [9]:
settings = combine_imps_wc.get_wf_defaults()
builder = combine_imps_wc.get_builder()
settings['jij_run']= True
settings['dos_run']= False
settings['lmdos']= False
settings['retrieve_kkrflex'] = False
settings['mag_init']= False
settings['threshold_switch_high_accuracy'] = 1e-2
settings['convergence_criterion'] = 1e-7 #convergence_criterion
settings['threshold_aggressive_mixing']= 0.02
settings['strmix']= 0.01
settings['aggrmix'] = 0.05
# settings['mixreduce'] = 0.5
settings['broyden-number'] = 20
settings['nsteps'] = 100
settings['kkr_runmax'] = 10

Version of workflow: 0.3.1


In [10]:
options = scf_node.inputs.options.get_dict()
options['max_wallclock_seconds'] = 36000*1
# options

In [11]:
builder.impurity1_output_node = impurity1_output_node
builder.impurity2_output_node = impurity2_output_node
builder.offset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
builder.scf.kkrimp = scf_node.inputs.kkrimp
builder.scf.options = Dict(dict=options)
builder.scf.wf_parameters = scf_node.inputs.wf_parameters

# host_gf namespace setup
builder.host_gf.kkr = host_gf.inputs.kkr
builder.host_gf.options = host_gf.inputs.options
builder.host_gf.params_kkr_overwrite = Dict(dict=kkr_flex_kkr_param) #host_gf.inputs.wf_parameters

builder.wf_parameters_overwrite = Dict(dict=settings)
builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)

In [12]:
# To submit the builder
combined_imp_submission = submit(builder)

In [17]:
# Note that here for doped 3d atoms into the Bi2Te3 host

group_74 = load_group(74)
## first node of group-64 is for kkr_imp_wc process
# is being used for the host_gf or kkr_flex files

imp_wc_with_gf = load_node(16100)
imp1_wc = group_74.nodes[1]
imp2_wc = group_74.nodes[2]

scf = imp1_wc

host_wf_parameters = Dict(dict={'NSHELD' : 2500})
host_kkr = Code().get_from_string('kkr@claix18_init')

scf_kkrimp = Code().get_from_string('kkrflex@claix18_init')
options = Dict(dict=metadata_option_1)

imp1_output_node = imp1_wc.outputs.workflow_info
imp2_output_node = imp2_wc.outputs.workflow_info
offset_imp2 = {'index':1}

settings = combine_imps_wc.get_wf_defaults()
settings['jij_run'] = True
settings['dos_run'] = False
settings['lmdos'] = False
settings['strmix'] = 0.01
settings['aggrmix'] = 0.01
settings['threshold_aggressive_mixing'] = 0.01
settings['convergence_criterion'] = 1e-7
settings['threshold_switch_high_accuracy'] = 1e-2
settings['retrieve_kkrflex'] = False
settings['mag_init'] = False
settings['mixreduce'] = 0.5
settings['nsteps'] = 100
settings['kkr_runmax'] = 10
settings['broyden-number'] = 20




combine_imp_builder = combine_imps_wc.get_builder()
combine_imp_builder.impurity1_output_node = imp1_output_node
combine_imp_builder.impurity2_output_node = imp2_output_node
combine_imp_builder.offset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
combine_imp_builder.scf.kkrimp = scf_kkrimp
combine_imp_builder.scf.options = options
combine_imp_builder.scf.wf_parameters = scf.inputs.wf_parameters

# combine_imp_builder.gf_host_remote = 
# host_gf namespace setup
combine_imp_builder.host_gf.kkr = host_kkr
combine_imp_builder.host_gf.options = options
combine_imp_builder.host_gf.params_kkr_overwrite = host_wf_parameters #host_gf.inputs.wf_parameters

combine_imp_builder.wf_parameters_overwrite = Dict(dict=settings)
combine_imp_builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)

# submission = submit(combine_imp_builder)

Version of workflow: 0.3.1


In [184]:
## Here trying to work with host Au and other impurity
## THe calc is taken from the wasmer
imp_Au_group = load_group(85)
wc_list = list(imp_Au_group.nodes)


imp_wc_1 = wc_list[0]
# print('first imp wc : ', imp_wc_1.label)
imp_wc_2 = wc_list[1]
# print('econd imp wc : ', imp_wc_2.label)
# setting the scf and host namespace

scf = imp_wc_1
host_wf_parameters = Dict(dict={'NSHELD' : 2500,
                                'BZDIVIDE': [40, 40, 40],
                                'FCM': 20,
                                'EMAX': 1.2,
                                'EMIN': -0.8,
                                'GMAX': 65.0,
                                'LMAX': 3,
                                'NCHEB': 12,
                              
                               })

host_kkr = Code().get_from_string('kkr@claix18_init')
scf_kkrimp = Code().get_from_string('kkrflex@claix18_init')
options = Dict(dict=metadata_option_1)

imp1_output_node = imp_wc_1.outputs.workflow_info
imp2_output_node = imp_wc_2.outputs.workflow_info
offset_imp2 = {'index':1}

settings = combine_imps_wc.get_wf_defaults()
settings['jij_run'] = False
settings['dos_run'] = False
settings['lmdos'] = False
settings['strmix'] = 0.001
settings['aggrmix'] = 0.008
settings['threshold_aggressive_mixing'] = 0.008
settings['convergence_criterion'] = 1e-7
settings['threshold_switch_high_accuracy'] = 1e-2
settings['retrieve_kkrflex'] = False
settings['mag_init'] = False
settings['mixreduce'] = 0.5
settings['nsteps'] = 100
settings['kkr_runmax'] = 10

combine_imp_builder = combine_imps_wc.get_builder()
combine_imp_builder.impurity1_output_node = imp1_output_node
combine_imp_builder.impurity2_output_node = imp2_output_node
combine_imp_builder.offset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
combine_imp_builder.scf.kkrimp = scf_kkrimp
combine_imp_builder.scf.options = options
combine_imp_builder.scf.wf_parameters = scf.inputs.wf_parameters

# host_gf namespace setup
combine_imp_builder.host_gf.kkr = host_kkr
combine_imp_builder.host_gf.options = options
combine_imp_builder.host_gf.params_kkr_overwrite = host_wf_parameters #host_gf.inputs.wf_parameters

combine_imp_builder.wf_parameters_overwrite = Dict(dict=settings)
combine_imp_builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)
# submission = submit(combine_imp_builder)

Version of workflow: 0.3.1


In [48]:
# flex = imp_wc_with_gf.get_outgoing(node_class=kkr_flex_wc).all()[0].node
# flex.get_outgoing(node_class=KkrCalculation).all()[0].node.inputs.parameters.get_dict()


In [40]:
Bi2Te3_conv_kkr = load_node('23b13d9d-6e02-46c7-b3ac-17033b205b19')
Bi2Te3_conv_kkr.pk

245

In [23]:
# Bi2Te3_conv_kkr.inputs.parameters.get_dict()
#

In [10]:
kkr_flex_kkr_param = kkr_calc.inputs.parameters.get_dict()
kkr_flex_kkr_param['RUNOPT'] = [kkr_flex_kkr_param['RUNOPT'][-1]]

In [14]:
kkr_flex_kkr_param.get

20.0

In [18]:
params = kkrparams